In [2]:
import pandas as pd
from pathlib import Path

In [6]:
n_sdg = 17
sdg_strings = [''.join(('sdg',str(ii))) for ii in range(1,n_sdg+1)]
folders = [d for d in Path('analysis_data').iterdir() if d.is_dir()]

In [9]:
# load all results
all_res = {}
for f in folders:
    orient = str(f).split(' - ')[1]
    all_res[orient] = {}
    for s in sdg_strings:
         path = list(f.glob('_'.join((s,'*'))))[0]
         all_res[orient][s] = pd.read_csv(path, sep='\t', index_col=0)

In [10]:
# save slacks in excel files
for orient in all_res.keys():
    path = Path('analysis_data')/''.join(('slacks_', orient, '.xlsx'))
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    for s in all_res[orient].keys():
        n_countries = all_res[orient][s].shape[0]
        all_res[orient][s].iloc[:,1:-n_countries].to_excel(writer, sheet_name=s)
    writer.save()

In [11]:
# save lambdas in excel files
for orient in all_res.keys():
    path = Path('analysis_data')/''.join(('lambdas_', orient, '.xlsx'))
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    for s in all_res[orient].keys():
        n_countries = all_res[orient][s].shape[0]
        all_res[orient][s].iloc[:,-n_countries:].to_excel(writer, sheet_name=s)
    writer.save()

In [12]:
# save efficiencies in csv files
for orient in all_res.keys():
    path = Path('analysis_data')/''.join(('eff_', orient, '.csv'))
    eff_df = pd.concat((all_res[orient][s]['eff'] for s in sdg_strings), axis=1) # read and concat
    eff_df.columns = sdg_strings
    eff_df.to_csv(path)
